
# ANN - Generative Adversarial Networks

* Edward Carrillo eccarrilloe@unal.edu.co
* Eliana Garzón egarzonv@unal.edu.co
* Diego Velásquez arvelasqueza@unal.edu.co
* Sneyder Gantiva esgantivar@unal.edu.co

In [ ]:
import numpy as np
import os 
import tensorflow as tf 
import matplotlib.pyplot as plt

In [ ]:
from keras.optimizers import Adam
from keras.datasets import mnist 
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input

In [ ]:
import imageio

## GAN architecture

Generative adversarial networks consist of two models: a generative model and a discriminative model.

<img src="https://ml-esgantivar.s3.amazonaws.com/ann-gan/GAN_Overall.png" style="width: 300px">

The discriminator model is a classifier that determines whether a given image looks like a real image from the dataset or like an artificially created image. This is basically a binary classifier that will take the form of a normal neural network.

The generator model takes random input values and transforms them into images through a deconvolutional neural network.

Over the course of many training iterations, the weights and biases in the discriminator and the generator are trained through backpropagation. The discriminator learns to tell "real" images of handwritten digits apart from "fake" images created by the generator. At the same time, the generator uses feedback from the discriminator to learn how to produce convincing images that the discriminator can't distinguish from real images.

## Discriminator network

Our discriminator is a  neural network that takes in an image of size 28 x 28 x 1 as input and returns a single scalar number that describes whether or not the input image is "real" or "fake"—that is, whether it's drawn from the set of MNIST images or generated by the generator.

<img src="https://ml-esgantivar.s3.amazonaws.com/ann-gan/GAN_Discriminator.png" style="width: 300px">
 

## Generator network

<img src="https://ml-esgantivar.s3.amazonaws.com/ann-gan/GAN_Generator.png" style="width: 300px">

You can think of the generator as a kind of reverse neural network. A typical NN like our discriminator network transforms a 2- or 3-dimensional matrix of pixel values into a single probability. A generator, however, takes a `d`-dimensional vector of noise and upsamples it to become a 28 x 28 image. ReLU and batch normalization are used to stabilize the outputs of each layer.

In [ ]:
class GAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False

        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            valid_y = np.array([1] * batch_size)
            g_loss = self.combined.train_on_batch(noise, valid_y)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()

In [ ]:
gan = GAN()
gan.train(epochs=30000, batch_size=32, save_interval=200)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
25000 [D loss: 0.648447, acc.: 59.38%] [G loss: 0.852660]
25001 [D loss: 0.720040, acc.: 43.75%] [G loss: 0.822275]
25002 [D loss: 0.685199, acc.: 53.12%] [G loss: 0.866822]
25003 [D loss: 0.645313, acc.: 62.50%] [G loss: 0.871826]
25004 [D loss: 0.675223, acc.: 56.25%] [G loss: 0.826977]
25005 [D loss: 0.658360, acc.: 65.62%] [G loss: 0.880102]
25006 [D loss: 0.642859, acc.: 78.12%] [G loss: 0.802947]
25007 [D loss: 0.653885, acc.: 62.50%] [G loss: 0.848913]
25008 [D loss: 0.695882, acc.: 53.12%] [G loss: 0.844388]
25009 [D loss: 0.622966, acc.: 65.62%] [G loss: 0.917784]
25010 [D loss: 0.709899, acc.: 50.00%] [G loss: 0.862660]
25011 [D loss: 0.667568, acc.: 59.38%] [G loss: 0.889329]
25012 [D loss: 0.695729, acc.: 62.50%] [G loss: 0.873212]
25013 [D loss: 0.709374, acc.: 53.12%] [G loss: 0.890880]
25014 [D loss: 0.669923, acc.: 59.38%] [G loss: 0.908630]
25015 [D loss: 0.660530, acc.: 62.50%] [G loss: 0.865198]
25016

In [ ]:
filenames = [f'/content/gan/images/mnist_{i}.png' for i in range(0, 30000, 200)]

In [ ]:
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('/content/gan/mnist_gan.gif', images, duration=0.4)

In [ ]:
!zip -r results.zip /content/gan/images

  adding: content/gan/images/ (stored 0%)
  adding: content/gan/images/mnist_4400.png (deflated 7%)
  adding: content/gan/images/mnist_27000.png (deflated 9%)
  adding: content/gan/images/mnist_21000.png (deflated 8%)
  adding: content/gan/images/mnist_15600.png (deflated 8%)
  adding: content/gan/images/mnist_26000.png (deflated 9%)
  adding: content/gan/images/mnist_29200.png (deflated 8%)
  adding: content/gan/images/mnist_27600.png (deflated 7%)
  adding: content/gan/images/mnist_800.png (deflated 8%)
  adding: content/gan/images/mnist_16000.png (deflated 8%)
  adding: content/gan/images/mnist_9200.png (deflated 7%)
  adding: content/gan/images/mnist_25600.png (deflated 8%)
  adding: content/gan/images/mnist_22800.png (deflated 8%)
  adding: content/gan/images/mnist_19800.png (deflated 8%)
  adding: content/gan/images/mnist_14000.png (deflated 8%)
  adding: content/gan/images/mnist_6600.png (deflated 8%)
  adding: content/gan/images/mnist_13000.png (deflated 8%)
  adding: content/g

## Results of training process

<img src="https://ml-esgantivar.s3.amazonaws.com/ann-gan/mnist_gan_res.gif">